In [2]:
library(DiffBind)
library(tidyverse)

In [31]:
wkdir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/3_ChIP/4_DiffBind'
setwd(wkdir)

In [32]:
samples.sheet <- '2017_PNAS_Li_simplified.csv'

dbObj <- dba(sampleSheet=samples.sheet)
dbObj <- dba.count(dbObj, bUseSummarizeOverlaps=TRUE)
dbObj <- dba.contrast(dbObj, categories=DBA_FACTOR, minMembers=2)
dbObj <- dba.analyze(dbObj, method=DBA_EDGER)
dba.show(dbObj, bContrasts=T)

# Write normalized rpkm dfs
for (i in c(1:4)) {
    i_name <- colnames(dbObj$class)[i]
    i_df <- dbObj$peaks[[i]]
    write_csv(i_df, paste(i_name, "norm.csv", sep="_"))
}

# Write differential outputs
res_edger <- dba.report(dbObj, method=DBA_EDGER, contrast = 1, th=1)
res_edger <- as.data.frame(res_edger)
res_edger$`p.value` <- NULL
res_edger$FDR <- NULL
write_csv(res_edger, "2017_PNAS_Li_CD8-IL2_DE.csv")

In [100]:
il2_df <- read_csv("STAT5B-IL2-CD8_1_norm.csv")
noil2_df <- read_csv("STAT5B-noIL2-CD8_1_norm.csv")

il2_df$noil2_RPKM <- noil2_df$RPKM
il2_df$noil2_Reads <- noil2_df$Reads
il2_df$log2FoldChange <- log2((il2_df$RPKM + 0.1) / (il2_df$noil2_RPKM + 0.1))

il2_df$noil2_RPKM <- noil2_df$RPKM
il2_df$noil2_Reads <- noil2_df$Reads
il2_df$log2FoldChange <- log2((il2_df$RPKM + 0.1) / (il2_df$noil2_RPKM + 0.1))

write_csv(il2_df, "2017_PNAS_Li_CD8-IL2_vs_CD8-noIL2_rpkmFoldChange.csv")


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Chr = col_double(),
  Start = col_double(),
  End = col_double(),
  Score = col_double(),
  RPKM = col_double(),
  Reads = col_double(),
  cRPKM = col_double(),
  cReads = col_double()
)


Warning message:
“179 parsing failures.
 row col expected                   actual                        file
3565 Chr a double CHR_CAST_EI_MMCHR11_CTG5 'STAT5B-IL2-CD8_1_norm.csv'
3566 Chr a double CHR_CAST_EI_MMCHR11_CTG5 'STAT5B-IL2-CD8_1_norm.csv'
3567 Chr a double CHR_CAST_EI_MMCHR11_CTG5 'STAT5B-IL2-CD8_1_norm.csv'
3568 Chr a double CHR_MG171_PATCH          'STAT5B-IL2-CD8_1_norm.csv'
3569 Chr a double CHR_MG184_PATCH          'STAT5B-IL2-CD8_1_norm.csv'
.... ... ........ ........................ ...........................
See problems(...) for more details.
”

── Column specification ─────────────────────────────────────────────────────────────